In [1]:
#########################################################################
#Project Name: RoboKart: Robot-assisted Grocery Shopping
#Authors Name: Anja Gross, Gunjan Gupta
#Date: 16-07-2020
#Machine Learning and Robotics Lab, University of Stuttgart, Germany
#########################################################################

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
#import the necessary libraries
import sys
sys.path.append('../../build')
import cv2 as cv
import numpy as np
import libry as ry
import time
import math
import PySimpleGUI as sg

In [3]:
#configuration file along with the configuration viewer
RealWorld = ry.Config()
RealWorld.addFile("../../scenarios/pandasTable_modified.g")
V = ry.ConfigurationViewer()
V.setConfiguration(RealWorld)

In [4]:
# Sets up the shelf on the left hand side of the robot 
l_shelf = RealWorld.addFrame("l_shelf")
l_shelf.setShape(ry.ST.ssBox, [0.3, 2., 0.6, 0.02])
l_shelf.setColor([0.6, 0.298, 0])
l_shelf.setPosition([-0.7, 0.0, 1.0])
    
# Sets up the shelf on the right hand side of the robot 
r_shelf = RealWorld.addFrame("r_shelf")
r_shelf.setShape(ry.ST.ssBox, [0.3, 2., 0.6, 0.02])
r_shelf.setColor([0.6, 0.298, 0])
r_shelf.setPosition([0.7, 0.0, 1.0])

#places the obstacle for collision detection
obstacle = RealWorld.addFrame("obstacle")
obstacle.setShape(ry.ST.ssBox, [0.1, 0.1, 0.5, 0.02])
obstacle.setPosition([0.2, 3.5, 1.0])
obstacle.setColor([1.0, 0.0, 0.0])

#objects available for shopping
itemNames = {
    
    "blue" : ["blue1","blue2","blue3","blue4"],
    "red" : ["red1", "red2","red3"],
    "green" : ["green1", "green2","green3"]
    }



In [5]:
#object creation
def createItem(name, color, pos):
    name = RealWorld.addFrame(name)
    colorVal = []
    
    if color == "blue":
        name.setShape(ry.ST.ssBox, [0.1, 0.1, 0.1, .02])
        colorVal = [0,0,1]
    elif color == "red":
        name.setShape(ry.ST.sphere, [0.05, 0.05, 0.05])
        colorVal = [1,0,0]
    elif color == "green":
        name.setShape(ry.ST.capsule, [0.05,0.06,0.04])
        colorVal = [0,1,0]

    name.setPosition(pos)
    name.setColor(colorVal) 
    
    return name

In [6]:
# Add cameras

# Right camera, looking at the right shelf
f_r_camera = RealWorld.frame("r_camera")

# Left camera, looking at the left shelf
f_l_camera = RealWorld.frame("l_camera")

# Cart camera, sitting in front of the cart and facing what is in front of the cart
f_cart_camera = RealWorld.frame("cart_camera")
V.setConfiguration(RealWorld)

S = RealWorld.simulation(ry.SimulatorEngine.physx, True)
S.addSensor("r_camera")
S.addSensor("l_camera")
S.addSensor("cart_camera")


In [7]:
# The camera parameters of all three cameras.
f = 0.895
f = f * 260.
fxfypxpy = [f, f, 320., 180.]

# save initial background
[rgb0, depth0] = S.getImageAndDepth()  #we don't need images with 100Hz, rendering is slow
points0 = S.depthData2pointCloud(depth0, fxfypxpy)

#creating empty objects, later used for grasping, in collision detection
obj = RealWorld.addFrame("object")
obj_obstacle = RealWorld.addFrame("objobstacle")
obj_gripper = RealWorld.addFrame("object_gripper")
targetObj = RealWorld.addFrame("targetdes")

In [8]:
#setting up the tau value that decides the smoothness of the robot motion
tau = 0.01
komo = RealWorld.komo_IK(False)

In [9]:
# method for opening the gripper for aligning and grasping the object
def openGripper():
    print('                  Opening                   ')
    print('--------------------------------------------')
    S.openGripper("L_gripper")
    while S.getGripperWidth("L_gripper") < 0.06: 
        time.sleep(tau)
        S.step([], tau, ry.ControlMode.none)

In [10]:
#method for aligning the gripper with respect to the orientation of the object for grasping and picking
def align(p_o, itemAlign, selCamera):
    print('                 Aligning                   ')
    print('--------------------------------------------')
    
    #align the gripper wrt to the cart for pushing it
    if itemAlign == "shopcart":
        p_o[1] = p_o[1] - 0.50
    for t in range(300):
        if S.getGripperIsGrasping("L_gripper"): break
        time.sleep(tau)
        q = S.get_q()
        obj.setPosition(p_o)

        #compute a target robot pose using optimization
        RealWorld.setJointState(q)
        V.setConfiguration(RealWorld)
        komo = RealWorld.komo_path(1.,1,tau, True)
        komo.clearObjectives()
        komo.add_qControlObjective(order = 1, scale = 1e0)
        komo.addObjective([], ry.FS.accumulatedCollisions, type = ry.OT.ineq, scale = [1e2])
        
        #align the gripper wrt to the object for grasping and picking it
        if itemAlign == "objects":
            komo.addObjective(type=ry.OT.eq, feature=ry.FS.scalarProductXX, frames=['L_gripperCenter', 'object'], target=[-1])
            komo.addObjective(type=ry.OT.eq, feature=ry.FS.scalarProductYY, frames=['L_gripperCenter', 'object'], target=[-1])
            komo.addObjective(type=ry.OT.eq, feature=ry.FS.scalarProductZZ, frames=['L_gripperCenter', 'object'], target=[1])
            komo.addObjective([], ry.FS.vectorZ, ["L_gripperCenter"], ry.OT.sos, scale=[1e1], target=[0,0,1])
            komo.addObjective(type=ry.OT.eq, feature=ry.FS.positionDiff, frames=['L_gripperCenter', 'object'], scale=[1e2], target=[0.0,0.0,0.2])

        else:  #Komo objectives to align with respect to the cart
            komo.addObjective(type=ry.OT.eq, feature=ry.FS.scalarProductXX, frames=['L_gripperCenter', 'object'], target=[0])
            komo.addObjective(type=ry.OT.eq, feature=ry.FS.scalarProductYY, frames=['L_gripperCenter', 'object'], target=[0])
            komo.addObjective(type=ry.OT.eq, feature=ry.FS.scalarProductZZ, frames=['L_gripperCenter', 'object'], target=[0])
            
            if selCamera == "lcam": #alignment for the cart wrt to the left camera
                komo.addObjective([], ry.FS.vectorX, ["L_gripperCenter"], ry.OT.eq, scale=[1e1], target=[0,0,-1])
            else: #alignment for the cart wrt to the right camera
                komo.addObjective([], ry.FS.vectorX, ["L_gripperCenter"], ry.OT.eq, scale=[1e1], target=[0,0,1])
            komo.addObjective(type=ry.OT.eq, feature=ry.FS.positionDiff, frames=['L_gripperCenter', 'object'], scale=[1e2], target=[0.0,-0.1,0.0])
            
        #keep gripper open
        komo.addObjective([], ry.FS.qItself, ["L_finger1"], ry.OT.eq, [1e1], order = 1)

        #add limit on velocity
        vel_lim = 100.
        komo.addObjective([], ry.FS.qItself, [], ry.OT.ineq, [1e1], target=[vel_lim]*8, order = 1)
        komo.addObjective([], ry.FS.qItself, [], ry.OT.ineq, [-1e1], target=[-vel_lim]*8, order = 1)

        komo.optimize()
        RealWorld.setFrameState(komo.getConfiguration(0))
        q = RealWorld.getJointState()

        #send controls to the simulation
        S.step(q, tau, ry.ControlMode.position)

    obj_gripper.setPosition(RealWorld.getFrame("L_gripperCenter").getPosition())

In [11]:
#method to grasp the objects 
def grasp(itemGrasp, selGraspCamera):
    print('                 Grasping                   ')
    print('--------------------------------------------')
    closing = False
    errPer, errCon = np.inf, np.inf
    while True:
        if S.getGripperIsGrasping("L_gripper"): break
        time.sleep(tau)
        q = S.get_q()
        
        #decide on grasp?
        if not closing:
            [y,J] = RealWorld.evalFeature(ry.FS.positionDiff, ["L_gripperCenter", "object"])
            errCon = np.abs(y).max()
            print(errCon)
            if errCon < 0.02:
                print('--------------------------------------------')
                print('       Closing the gripper                  ')
                print('--------------------------------------------')
                closing = True
                S.closeGripper("L_gripper")

        #compute a target robot pose using optimization
        RealWorld.setJointState(q)
        V.setConfiguration(RealWorld)
        komo = RealWorld.komo_path(1.,1,tau, True)
        komo.clearObjectives()
        komo.add_qControlObjective(order = 1, scale = 1e0)
        komo.addObjective([], ry.FS.accumulatedCollisions, type = ry.OT.ineq, scale = [1e2])
        
        if itemGrasp == "objects": #komo objectives to grasp the objects
            komo.addObjective(type=ry.OT.eq, feature=ry.FS.scalarProductXX, frames=['L_gripperCenter', 'object'], target=[-1])
            komo.addObjective(type=ry.OT.eq, feature=ry.FS.scalarProductYY, frames=['L_gripperCenter', 'object'], target=[-1])
            komo.addObjective(type=ry.OT.eq, feature=ry.FS.scalarProductZZ, frames=['L_gripperCenter', 'object'], target=[1])
            komo.addObjective([], ry.FS.vectorZ, ["L_gripperCenter"], ry.OT.eq, scale=[1e1], target=[0,0,1])
            komo.addObjective(type=ry.OT.eq, feature=ry.FS.positionDiff, frames=['L_gripperCenter', 'object'], scale=[1e2], target=[0.0,0.0,0.02])
        else: #komo objectives to grasp the shopping cart
            komo.addObjective(type=ry.OT.eq, feature=ry.FS.scalarProductXX, frames=['L_gripperCenter', 'object'], target=[0])
            komo.addObjective(type=ry.OT.eq, feature=ry.FS.scalarProductYY, frames=['L_gripperCenter', 'object'], target=[0])
            komo.addObjective(type=ry.OT.eq, feature=ry.FS.scalarProductZZ, frames=['L_gripperCenter', 'object'], target=[0])
            
            if selGraspCamera == "lcam": #objectives for the cart wrt to the left camera (when objects are only picked from the left shelf)
                komo.addObjective([], ry.FS.vectorX, ["L_gripperCenter"], ry.OT.eq, scale=[1e1], target=[0,0,-1])
            else: #objectives for the cart wrt to the right camera (when objects are picked from the right shelf)
                komo.addObjective([], ry.FS.vectorX, ["L_gripperCenter"], ry.OT.eq, scale=[1e1], target=[0,0,1])
            komo.addObjective(type=ry.OT.eq, feature=ry.FS.positionDiff, frames=['L_gripperCenter', 'object'], scale=[1e2], target=[0.0,0.02,0.0])
        #keep gripper open
        komo.addObjective([], ry.FS.qItself, ["L_finger1"], ry.OT.eq, [1e1], order = 1)

        #add limit on velocity
        vel_lim = 100.
        komo.addObjective([], ry.FS.qItself, [], ry.OT.ineq, [1e1], target=[vel_lim]*8, order = 1)
        komo.addObjective([], ry.FS.qItself, [], ry.OT.ineq, [-1e1], target=[-vel_lim]*8, order = 1)

        komo.optimize()
        RealWorld.setFrameState(komo.getConfiguration(0))
        q = RealWorld.getJointState()

        #send controls to the simulation
        S.step(q, tau, ry.ControlMode.position)

In [12]:
#method to lift the object after grasping from the shelf
def lift():
    print('                Lifting                     ')
    print('--------------------------------------------')
    errPer, errCon = np.inf, np.inf
    
    for t in range(100):
        time.sleep(tau)
        q = S.get_q()

        #compute a target robot pose using optimization
        RealWorld.setJointState(q)
        V.setConfiguration(RealWorld)
        komo = RealWorld.komo_path(1.,1,tau, True)
        komo.clearObjectives()
        komo.add_qControlObjective(order = 1, scale = 1e0)
        #komo objectives for lifting the objects (from either shelf)
        komo.addObjective([], ry.FS.accumulatedCollisions, type = ry.OT.ineq, scale = [1e2])
        komo.addObjective(type=ry.OT.eq, feature=ry.FS.scalarProductXX, frames=['L_gripperCenter', 'object_gripper'], target=[-1])
        komo.addObjective(type=ry.OT.eq, feature=ry.FS.scalarProductYY, frames=['L_gripperCenter', 'object_gripper'], target=[-1])
        komo.addObjective(type=ry.OT.eq, feature=ry.FS.scalarProductZZ, frames=['L_gripperCenter', 'object_gripper'], target=[1])
        komo.addObjective([], ry.FS.vectorZ, ["L_gripperCenter"], ry.OT.eq, scale=[1e1], target=[0,0,1])
        komo.addObjective(type=ry.OT.eq, feature=ry.FS.positionDiff, frames=['L_gripperCenter', 'object_gripper'], scale=[1e2], target=[0.0,0.0,0.1])

        #keep gripper open
        komo.addObjective([], ry.FS.qItself, ["L_finger1"], ry.OT.eq, [1e1], order = 1)

        #add limit on velocity
        vel_lim = 100.
        komo.addObjective([], ry.FS.qItself, [], ry.OT.ineq, [1e1], target=[vel_lim]*8, order = 1)
        komo.addObjective([], ry.FS.qItself, [], ry.OT.ineq, [-1e1], target=[-vel_lim]*8, order = 1)

        komo.optimize()
        RealWorld.setFrameState(komo.getConfiguration(0))
        q = RealWorld.getJointState()

        #send controls to the simulation
        S.step(q, tau, ry.ControlMode.position)

In [13]:
#method to truncate the digits to obtain the position of the object/obstacle (obtained using perception) upto the specified digits
def truncate(num, digits):
    stepper = 10.0 ** digits
    return math.trunc(stepper * num)/stepper

In [14]:
#method to place the objects on the cart after lifting them
def place(num):
    print('                Placing                     ')
    print('--------------------------------------------')
    
    #specify the position of placing on the cart based on the color, so that objects of the same color are placed together
    if num == 0:
        targetObj.setPosition([0.20, 0.35, 1.4]) #position of red object on the cart
    elif num == 1:
        targetObj.setPosition([0.20, 0.20, 1.6]) #position of green object on the cart
    else:
        targetObj.setPosition([-0.35, 0.40, 1.6]) #position of blue object on the cart

    for t in range(100):
        time.sleep(tau)            
        q = S.get_q()

        #compute a target robot pose using optimization
        RealWorld.setJointState(q)
        V.setConfiguration(RealWorld)
        komo = RealWorld.komo_path(1.,1,tau, True)
        komo.clearObjectives()
        komo.add_qControlObjective(order = 1, scale = 1e0)
        #komo objectives for placing the objects on the cart
        komo.addObjective([], ry.FS.accumulatedCollisions, type = ry.OT.ineq, scale = [1e2])
        komo.addObjective(type=ry.OT.eq, feature=ry.FS.scalarProductXX, frames=['L_gripperCenter', 'targetdes'], target=[-1])
        komo.addObjective(type=ry.OT.eq, feature=ry.FS.scalarProductYY, frames=['L_gripperCenter', 'targetdes'], target=[-1])
        komo.addObjective(type=ry.OT.eq, feature=ry.FS.scalarProductZZ, frames=['L_gripperCenter', 'targetdes'], target=[1])
        komo.addObjective([], ry.FS.vectorZ, ["L_gripperCenter"], ry.OT.eq, scale=[1e1], target=[0,0,1])
        komo.addObjective(type=ry.OT.eq, feature=ry.FS.positionDiff, frames=['L_gripperCenter', 'targetdes'], scale=[1e2], target=[0.0,0.0,0.0])

        #keep gripper open
        komo.addObjective([], ry.FS.qItself, ["L_finger1"], ry.OT.eq, [1e1], order = 1)

        #add limit on velocity
        vel_lim = 100.
        komo.addObjective([], ry.FS.qItself, [], ry.OT.ineq, [1e1], target=[vel_lim]*8, order = 1)
        komo.addObjective([], ry.FS.qItself, [], ry.OT.ineq, [-1e1], target=[-vel_lim]*8, order = 1)

        komo.optimize()
        RealWorld.setFrameState(komo.getConfiguration(0))
        q = RealWorld.getJointState()

        #send controls to the simulation
        S.step(q, tau, ry.ControlMode.position)

In [15]:
#function to display the color of the items on the shelf as a list
def cvfunc(colorlist):
    cols=[]
    templist= 0
    for i in range(len(colorlist)):
        j=0
        templist=colorlist[i]
        while j< templist:
            if i == 0:
                cols.append("red")
            elif i==1:
                cols.append("green")
            else:
                cols.append("blue")
    
            j = j+ 1
    return cols    
        

In [16]:
#the main method responsible for shoppping (all logic written in this method)
def shopping(itemCol, itemNum, maxItems):
    print('--------------------------------------------')
    print('                 Shopping                   ')
    temp = 0
    flagleft= False
    flagright= False
    redundant_items= 0
    robotPos = "center"  #initially the robot's position will be at the center
    #to check if the shopping cart is empty
    if maxItems==0:
        print('    Please add items into the cart          ')
        print('--------------------------------------------')
        sg.Popup("Cart is empty!",font=("ComicSansMS", 12),button_color=('Black', 'White'),button_type=4)
    else:
        #the robot first moves to the left shelf
        moveRobot1(-0.25)
        robotPos = "left"
        initialLeftJointState = RealWorld.getJointState()
        #color segmentation to display the object colors on the left shelf as a list 
        colList = colorSeg("lcam")
        cols = cvfunc(colList)
        print('     Starting with the Left shelf           ')
        print('--------------------------------------------')

        # For Left Shelf
        for k in range(len(itemCol)):
            flagleft = False
            if itemNum[k] != 0:
                for j in range(len(cols)):
                    if itemCol[k] == cols[j]:
                        flagleft= True
                        #checking for the red objects
                        if cols[j] == "red":
                            itemColor[0] = "red"
                            itemcount = 0
                            #responsible for Perception to provide the position (x-y-z) coorindates of the object that needs to be shopped.
                            doItemPerception("camera_l")
                            #method to get the coordinates of the perceived object from the left camera
                            arr = getObjPointPos("camera_l", "red")
                            m = len(arr)
                            while itemcount < m:
                                arr[itemcount] = truncate(arr[itemcount],3)   
                                itemcount = itemcount + 1
                            #adding offset to the perceived position
                            arr[0]= arr[0] + 0.069 # along x axis
                            arr[1]= arr[1] + 0.010 # along y axis
                            arr[2]= arr[2] - 0.030 # along z axis
                            p_obj = arr
                            number = 0
                        #checking for the green objects
                        elif cols[j] == "green":
                            itemColor[0] = "green"
                            itemcount = 0
                            doItemPerception("camera_l")
                            arr = getObjPointPos("camera_l", "green")
                            m = len(arr)
                            while itemcount < m:
                                arr[itemcount] = truncate(arr[itemcount],3)   
                                itemcount = itemcount + 1
                            arr[0]= arr[0] + 0.069 # along x axis
                            arr[1]= arr[1] + 0.010 # along y axis
                            arr[2]= arr[2] + 0.014 # along z axis
                            p_obj = arr
                            number = 1
                        #checking for the blue objects
                        else:
                            itemColor[0] = "blue"
                            itemcount = 0
                            doItemPerception("camera_l")
                            arr = getObjPointPos("camera_l", "blue")
                            m = len(arr)
                            while itemcount < m:
                                arr[itemcount] = truncate(arr[itemcount],3)   
                                itemcount = itemcount + 1
                            arr[0]= arr[0] + 0.069 # along x axis
                            arr[1]= arr[1] + 0.13 # along y axis
                            arr[2]= arr[2] - 0.023 # along z axis
                            p_obj = arr
                            number = 2

                        openGripper()
                        align(p_obj, "objects", "lcam")
                        grasp("objects", "lcam")
                        lift()
                        place(number)
                        openGripper()
                        RealWorld.setJointState(initialLeftJointState) #this makes the robot to go back to its initial joint state after placing each object on the cart
                        itemNum[k] = itemNum[k] - 1
                        temp = temp + 1
                    #to check if shopping is completed for a particular color-object before proceeding to the right shelf
                    if itemNum[k] == 0:
                        print('Shopping is done for {}' .format(itemCol[k]))
                        print('-------------------------------------------')
                        break
                #to check if an object of a different color (other then red, blue or green) is desired to be shopped- through the user input from GUI
                if flagleft== False:
                    print('Item {} not found in the left shelf!'.format(itemCol[k])) #to print on the console
                    sg.Popup("Item unavailable on the left shelf!",font=("ComicSansMS", 12),button_color=('Black', 'White'),button_type=4)
        #RealWorld.setJointState(initialLeftJointState)
        print('             Left shelf done              ')
        print('------------------------------------------')
        if temp != maxItems:  # To check whether we need to go to right shelf or not
            print('       Continuing with the Right shelf      ')
            #after the robot has picked all the objects it could have from the left shelf, it moves to the right shelf and performs the same functionality as above
            moveRobot1(0.50)
            robotPos = "right"
            initialRightJointState = RealWorld.getJointState()

        while temp != maxItems:  # To check whether we need to go to right shelf or not
            print('       Continuing with the Right shelf      ')
            print('--------------------------------------------')
            colList= colorSeg("rcam")
            cols = cvfunc(colList)
            for k in range(len(itemCol)):
                flagright= False
                if itemNum[k] != 0:
                    for j in range(len(cols)):
                        if itemCol[k] == cols[j]:
                            flagright = True
                            #checking for red objects
                            if cols[j] == "red":
                                itemColor[0] = "red"
                                itemcount = 0
                                doItemPerception("camera_r")
                                #method to get the coordinates of the perceived object from the right camera
                                arr = getObjPointPos("camera_r", "red")
                                m = len(arr)
                                while itemcount < m:
                                    arr[itemcount] = truncate(arr[itemcount],3)   
                                    itemcount = itemcount + 1
                                arr[0]= arr[0] - 0.069 # along x axis
                                arr[1]= arr[1] + 0.03 # along y axis
                                arr[2]= arr[2] - 0.033 # along z axis
                                p_obj = arr
                                number = 0
                            #checking for green objects
                            elif cols[j] == "green":
                                itemColor[0] = "green"
                                itemcount = 0
                                doItemPerception("camera_r")
                                arr = getObjPointPos("camera_r", "green")
                                m = len(arr)
                                while itemcount < m:
                                    arr[itemcount] = truncate(arr[itemcount],3)   
                                    itemcount = itemcount + 1
                                arr[0]= arr[0] - 0.069 # along x axis
                                arr[1]= arr[1] + 0.130 # along y axis
                                arr[2]= arr[2] + 0.014 # along z axis
                                p_obj = arr
                                number = 1
                            #checking for the blue objects
                            else:
                                itemColor[0] = "blue"
                                itemcount = 0
                                doItemPerception("camera_r")
                                arr = getObjPointPos("camera_r", "blue")
                                m = len(arr)
                                while itemcount < m:
                                    arr[itemcount] = truncate(arr[itemcount],3)   
                                    itemcount = itemcount + 1
                                arr[0]= arr[0] - 0.069 # along x axis
                                arr[1]= arr[1] + 0.030 # along y axis
                                arr[2]= arr[2] - 0.030 # along z axis
                                p_obj = arr
                                number = 2

                            openGripper()
                            align(p_obj, "objects", "rcam")
                            grasp("objects", "rcam")
                            lift()
                            place(number)
                            openGripper()
                            RealWorld.setJointState(initialRightJointState)
                            itemNum[k] = itemNum[k] - 1
                            temp = temp + 1

                        #to check if shopping is completed for a particular color-object after picking the objects from the right shelf
                        if itemNum[k] == 0:
                            print('Shopping is done for {}' .format(itemCol[k]))
                            print('-------------------------------------------')
                            break
                     #to check if an object of a different color (other then red, blue or green) is desired to be shopped- through the user input from GUI
                    if flagright== False:
                        print('Item {} not found in the right shelf!'.format(itemCol[k])) #to display on the terminal.
                        sg.Popup("Item unavailable on the right shelf!",font=("ComicSansMS", 12),button_color=('Black', 'White'),button_type=4)
                        temp=temp + itemNum[k]
                        redundant_items = itemNum[k]

        print('             Right shelf done               ')
        print('--------------------------------------------')
#         if robotPos == "right":
#             RealWorld.setJointState(initialRightJointState)

    # writing the updated list into the file
    for i in range(0, len(itemNum)):
        itemNum[i] = str(itemNum[i])
    itemNum= [' {0} '.format(elem)for elem in itemNum]
    list= np.char.add(itemCol, itemNum)

    with open('shoppingList.txt','w') as f:
        for item in list:
            f.write("%s\n" % item)

    #cart motion started after the shopping is completed and all objects to be picked are placed on the cart.
    if(maxItems-redundant_items!=0):
        print('             Cart Motion started            ')
        print('--------------------------------------------')
        p_obj = RealWorld.getFrame("cart").getPosition()
        if robotPos == "left":
            posCam = "lcam"
        elif robotPos == "right":
            posCam = "rcam"
        else:
            print("Cart position is not proper to push")
        openGripper()
        align(p_obj,"shopcart", posCam)
        grasp("shopcart", posCam)
        pushCart()
        print('             Shopping completed             ')
        print('--------------------------------------------')


In [17]:
#method for robot motion using step function
def moveRobot1(limit):
    print('--------------------------------------------')
    print('                 Moving Robot               ')
    print('--------------------------------------------')
    linkPos = RealWorld.frame("L_panda_link0")
    linkTemp = linkPos.getPosition()
    linkPos.setPosition([linkTemp[0]+limit, linkTemp[1], linkTemp[2]])
    q = S.get_q()
    S.step(q, tau, ry.ControlMode.position)

In [18]:
# method for robot to push the cart after it aligns and grasps its gripper wrt to the cart
def pushCart():
    print('            Pushing the cart now!           ')
    print('--------------------------------------------')
    nearing = False
    end = False
    linkPos = RealWorld.frame("L_panda_link0")
    cartPos = RealWorld.frame("cart")
    #the camera on the cart perceives any obstacle, if any along the robot's path 
    doItemPerception("camera_cart")
    #method to get the coordinates of the obstacle perceived from the cart camera
    arr= getObstaclePos()
    m = len(arr)
    itemcount = 0
    while itemcount < m:
        arr[itemcount] = truncate(arr[itemcount],1)   
        itemcount = itemcount + 1
    obj_obstacle.setPosition(arr)
    while True:
        time.sleep(tau)
        linkTemp = linkPos.getPosition()
        #obstacle detection
        if not nearing:
            [y,J] = RealWorld.evalFeature(ry.FS.positionDiff, ["cart", "objobstacle"])
            errCon = np.abs(y).max()
            if errCon < 0.70: 
                print('            Obsatcle Detection!           ')
                print('------------------------------------------')
                nearing = True
                #collision avoidance by altering the robot path to turn to the left after detecting an obstacle.
                pushLeftCart()
                linkTemp = linkPos.getPosition()
       #end position for the robot to stop wrt to the table end position
        if not end:
            [y,J] = RealWorld.evalFeature(ry.FS.positionDiff, ["cart", "table"])
            errCon1 = np.abs(y).max()
            if errCon1 > 2.5: 
                print('            The End!                      ')
                print('------------------------------------------')
                end = True
                break

        linkPos.setPosition([linkTemp[0], linkTemp[1]+0.0005, linkTemp[2]])
        q = S.get_q()
        S.step(q, tau, ry.ControlMode.position)

In [19]:
# method for the robot to shift the position of the cart to left, after it detects an obstacle
def pushLeftCart():
    print('            Avoiding Obstacle!            ')
    print('------------------------------------------')

    linkPos = RealWorld.frame("L_panda_link0")

    for t in range(1000):
        time.sleep(tau)
        linkTemp = linkPos.getPosition()
        linkPos.setPosition([linkTemp[0]-0.0005, linkTemp[1], linkTemp[2]])
        q = S.get_q()
        S.step(q, tau, ry.ControlMode.position)

In [20]:
#this block includes the working of perception- to obtain the position of the objects on the shelf or the cart real-time.
objPos_r = np.zeros(shape=(360,640,3)) #position of the object on the left shelf
objPos_l = np.zeros(shape=(360,640,3)) #position of the object on the right shelf
objPos_cart = np.zeros(shape=(360,640,3)) #position of the obstacle wrt to the cart camera

itemColor = [0] * 1
#default color
itemColor[0] = ""

objPosPoint = np.zeros(3)
objPosPoint_r = np.zeros(3)
objPosPoint_l = np.zeros(3)
objPosPoint_cart = np.zeros(3)
objPoints_r = None
objPoints_l = None

bgr = None


# Retrieves the mean point of all the points of the most right positioned object of a given colour in the given camera image (x, y, z coordinates) in world frame.
def getObjPointPos(camera, colorName):
    itemColor[0] = colorName

    if camera == "camera_r":
        return objPosPoint_r
    elif camera == "camera_l":
        return objPosPoint_l

    
# Retrieves the mean point of all the points of the obstacle in front of the cart in the picture of the cart camera (x, y, z coordinates) in world frame
def getObstaclePos():
    return objPosPoint_cart

# Gets the images of the right, the left and the cart camera.
# Sets the mean position of all the points of the most right object of the current itemColor to the left and the right camera.
# Sets the mean position of all the points of the obstacle.
def doItemPerception(camera):

    points = []
    tau = .01

    fertig = False  

    for t in range(100):
        time.sleep(0.01)

        #grab sensor readings from the simulation
        q = S.get_q()
        if t%10 == 0:
            if camera == "camera_r":
                # Images from right camera
                S.selectSensor("r_camera")
                [rgb, depth] = S.getImageAndDepth()
                #global bgr
                bgr = cv.cvtColor(rgb, cv.COLOR_RGB2BGR)
                hsv = cv.cvtColor(bgr, cv.COLOR_BGR2HSV)          

                # Gets the points of the depth picture of the selected camera
                points = S.depthData2pointCloud(depth, fxfypxpy)
                
                # This is accessing the global objPos_r variable! Otherwise it would create a local one instead
                global objPosPoint_r
                masked, p_obj_rel = findObjPointPos(hsv, depth)

                # Cooordinate conversion from camera frame to world frame
                obj = RealWorld.addFrame("object", "r_camera")
                obj.setRelativePosition(p_obj_rel)
                p_obj_world = obj.getPosition()
                objPosPoint_r = p_obj_world
                
                 # Attach the point cloud to the rbg/bgr image (IMPORTANT: can only be seen in configuration space,
                 # where rgb holds vs. in cv bgr is used. So blue and red are swapped!)            
                 #r_cameraFrame.setPointCloud(objPosPoint_r, bgr)
           
            
            elif camera == "camera_l":
                # Images from left camera
                S.selectSensor("l_camera")
                [rgb, depth] = S.getImageAndDepth()
                bgr = cv.cvtColor(rgb, cv.COLOR_RGB2BGR)
                hsv = cv.cvtColor(bgr, cv.COLOR_BGR2HSV)

                global objPosPoint_l
                masked, p_obj_rel = findObjPointPos(hsv, depth)

                # Coordinate conversion from camera frame to world frame
                obj = RealWorld.addFrame("object", "l_camera")
                obj.setRelativePosition(p_obj_rel)
                p_obj_world = obj.getPosition()
                objPosPoint_l = p_obj_world
                
            else:
                #images from the cart camera
                S.selectSensor("cart_camera")
                [rgb, depth] = S.getImageAndDepth()
                bgr = cv.cvtColor(rgb, cv.COLOR_RGB2BGR)
                hsv = cv.cvtColor(bgr, cv.COLOR_BGR2HSV)


                global objPosPoint_cart
                masked, p_obj_rel = findObstaclePoints(hsv, depth)

                # Coordinate conversion from camera frame to world frame
                obj = RealWorld.addFrame("object", "cart_camera")
                obj.setRelativePosition(p_obj_rel)
                p_obj_world = obj.getPosition()
                objPosPoint_cart = p_obj_world
            

            V.recopyMeshes(RealWorld)
            V.setConfiguration(RealWorld)

            #if len(bgr)>0: cv.imshow("OPENCV - bgr", bgr) #to print the bgr contours
            #if len(hsv)>0: cv.imshow("OPENCV - hsv", hsv) #to print the hsv contours
            if len(masked)>0: cv.imshow("OPENCV - mask", masked) # to print the mask

            if cv.waitKey(1) & 0xFF == ord("q"): break
        S.step([], tau, ry.ControlMode.none)


# Gets the mean position of the most right positioned object in the camera image (given the defined color in "itemColor")
# and relative to the camera frame!.
def findObjPointPos(hsv, depth):
    x_value = None
    y_value = None
    mask = None

    # Creates thresholds for the colour red based on that a mask of all the red objects in the given camera image.
    if itemColor[0] == "red":
        lower_red1 = np.array([0,50,20])
        upper_red1 = np.array([5,255,255])
        lower_red2 = np.array([175,50,20])
        upper_red2 = np.array([180,255,255])

        hue = hsv[:,:,0]
        sat = hsv[:,:,1]
        val = hsv[:,:,2]
        
        # Get all red pixels with their x and y values
        pixel_is_red = np.logical_and(np.logical_or(hue < 16, hue > 164), sat > 150, val > 150)
        x_value, y_value = np.where(pixel_is_red)
        
        # Create a mask (an image where all red pixels are colored white and all the others black)
        mask1 = cv.inRange(hsv, lower_red1, upper_red1)
        mask2 = cv.inRange(hsv, lower_red2, upper_red2)
        mask = mask1 + mask2
    
    # Creates thresholds for the colours blue or green based on that a mask of all the blue (or green) objects in the given camera image.
    elif itemColor[0] == "blue":
        # define range of blue colour in HSV
        lower_blue = np.array([110,50,50])
        upper_blue = np.array([130,255,255])
        
        # Create a mask (an image where all color pixels are colored white and all the others black)
        mask = cv.inRange(hsv, lower_blue, upper_blue)
        
    elif itemColor[0] == "green":
        # define range of green colour in HSV
        lower_green = np.array([25,52,72])
        upper_green = np.array([102,255,255])
        
        # Create a mask (an image where all color pixels are colored white and all the others black)
        mask = cv.inRange(hsv, lower_green, upper_green)
    
    else:
        print("Wrong Item Color")

    # create empty mask
    filtered_mask = np.zeros(mask.shape, np.uint8)
    # find contours
    contours, _ = cv.findContours(mask, cv.RETR_LIST, cv.CHAIN_APPROX_SIMPLE)
    # Draws the contours (i.e. the complete area) of the object with index 0 in contours list, i.e. the object that is the most right in the image.
    cv.drawContours(filtered_mask, contours, 0, (255,255,255),-1)
    # Get the point cloud of the object in the mask in image coordinates
    objPixelPointCloud = image_to_pointcloud(depth, filtered_mask)
    # Transforms the pount cloud (pixel points) into meters (but still relative to the camera frame)
    obj_points = meter_pointcloud(objPixelPointCloud, fxfypxpy)
    # Computes the mean
    p_obj_rel = np.mean(obj_points, axis=0)

    return mask, p_obj_rel

# Gets the point cloud of the object in image coordinates.
# Retrieves an array containing the x, y coordinates from the object mask and the corresponding x, y coordinates (depth-meters) from the depth image 
def image_to_pointcloud(depth, mask):
    mask_pixels = np.where(mask>0)
    pointcloud = np.empty((mask_pixels[0].shape[0], 3))
    pointcloud[:,0] = mask_pixels[1]  # x pixels
    pointcloud[:,1] = mask_pixels[0]  # y pixels
    pointcloud[:,2] = depth[mask_pixels[0], mask_pixels[1]]
    return pointcloud

# Transforms the point cloud (pixel points from the camera image) into meters with the camera preferences (to get the real point cloud positions in 3D space).
def meter_pointcloud(pixel_points, fxfypxpy):
    points = np.empty(np.shape(pixel_points))
    for i, p in enumerate(pixel_points):
        x = p[0]
        y = p[1]
        d = p[2]
        
        px = fxfypxpy[-2]
        py = fxfypxpy[-1]
        
        x_ =  d * (x-px) / fxfypxpy[0]
        y_ = -d * (y-py) / fxfypxpy[1]
        z_ = -d
        points[i] = [x_,y_,z_]
    return points

# Retrieves the mean point of all the object points of the obstacle (relative to the camera frame!).
# This only uses red color detection, so the obstacle needs to be red in order to detect it!
def findObstaclePoints(hsv, depth):
    lower_red1 = np.array([0,50,20])
    upper_red1 = np.array([5,255,255])
    lower_red2 = np.array([175,50,20])
    upper_red2 = np.array([180,255,255])

    hue = hsv[:,:,0]
    sat = hsv[:,:,1]
    val = hsv[:,:,2]

    pixel_is_red = np.logical_and(np.logical_or(hue < 16, hue > 164), sat > 150, val > 150)
    x_value, y_value = np.where(pixel_is_red)
    
    # Create a mask (an image where all red pixels are colored white and all the others black)
    mask1 = cv.inRange(hsv, lower_red1, upper_red1)
    mask2 = cv.inRange(hsv, lower_red2, upper_red2)
    mask = mask1 + mask2
    
    # create empty mask
    filtered_mask = np.zeros(mask.shape, np.uint8)
    # find contours
    contours, _ = cv.findContours(mask, cv.RETR_LIST, cv.CHAIN_APPROX_SIMPLE)
    # Draws the contours (i.e. the complete area) of the obstacle.
    cv.drawContours(filtered_mask, contours, 0, (255,255,255),-1)
    # Get the point cloud of the object in the mask in image coordinates
    objPixelPointCloud = image_to_pointcloud(depth, filtered_mask)
    # Transforms the pount cloud (pixel points) into meters (but still relative to the camera frame)
    obj_points = meter_pointcloud(objPixelPointCloud, fxfypxpy)
    # Computes the mean
    p_obj_rel = np.mean(obj_points, axis=0)

    return mask, p_obj_rel


In [21]:
#method for color-segmentation for each-shelf, to return the object colors as a lsit (input for the method-Shopping())
def colorSeg(selCamera):
    points = []
    colorList = [0] * 3  # order of the colorlist is RGB
    colors = []
    font = cv.FONT_HERSHEY_COMPLEX
    
    if selCamera == "lcam":
        S.addSensor("l_camera")
    else:
        S.addSensor("r_camera")

    [rgb, depth] = S.getImageAndDepth()

    hsvFrame = cv.cvtColor(rgb, cv.COLOR_RGB2HSV) 
    
    
    # define range of blue colour in HSV
    blue_lower = np.array([110,50,50], np.uint8) 
    blue_upper = np.array([130, 255, 255], np.uint8) 
    blue_mask = cv.inRange(hsvFrame, blue_lower, blue_upper) 
    
    # define range of green colour in HSV
    green_lower = np.array([25, 52, 72], np.uint8) 
    green_upper = np.array([102, 255, 255], np.uint8) 
    green_mask = cv.inRange(hsvFrame, green_lower, green_upper) 
    
    # define range of red colour in HSV
    red_lower = np.array([0, 50, 50], np.uint8) 
    red_upper = np.array([10, 255, 255], np.uint8) 
    red_mask = cv.inRange(hsvFrame, red_lower, red_upper) 
    kernal = np.ones((5, 5), "uint8") 
    
    
    #creating masks of blue color
    blue_mask = cv.dilate(blue_mask, kernal) 
    res_blue = cv.bitwise_and(rgb, rgb,  
                              mask = blue_mask) 
    #creating masks of green color
    green_mask = cv.dilate(green_mask, kernal) 
    res_green = cv.bitwise_and(rgb, rgb, 
                                mask = green_mask) 
    #creating masks of red color
    red_mask = cv.dilate(red_mask, kernal) 
    res_red = cv.bitwise_and(rgb, rgb, 
                               mask = red_mask)

    # Creating contour to track blue color 
    contours, hierarchy = cv.findContours(blue_mask, 
                                           cv.RETR_TREE, 
                                           cv.CHAIN_APPROX_SIMPLE) 
    for pic, contour in enumerate(contours): 
        colorList[2]= colorList[2] + 1


    # Creating contour to track green color 
    contours, hierarchy = cv.findContours(green_mask, 
                                           cv.RETR_TREE, 
                                           cv.CHAIN_APPROX_SIMPLE) 
    for pic, contour in enumerate(contours): 
        colorList[1]= colorList[1] + 1

    # Creating contour to track red color 
    contours, hierarchy = cv.findContours(red_mask, 
                                           cv.RETR_TREE, 
                                           cv.CHAIN_APPROX_SIMPLE) 
    #based on the colors of the contours, adding the colors into a list, to determine the order of the objects on the shelf
    for pic, contour in enumerate(contours): 
        colorList[0]= colorList[0] + 1

    return colorList
    

In [22]:
def destroy():
        S = 0
        RealWorld = 0
        V = 0      

In [ ]:
numbers = [0]*4
#GUI for user interaction
sg.theme('DefaultNoMoreNagging')   # Add a touch of color
background = '#F0F0F0'
# All the stuff inside the window.
layout = [  [sg.Text('Welcome to RoboKart: Robot-Assisted shopping experience!',font=("ComicSansMS", 12))],
            [sg.Text('Please fill in the shopping cart!',font=("ComicSansMS", 12))],
            [sg.Button('Green:', size=(8, 1), button_color=('Black', 'Green'),font=("ComicSansMS", 12)), sg.InputText()],
            [sg.Button('Blue:', size=(8, 1),button_color=('Black', 'Blue'),font=("ComicSansMS", 12)), sg.InputText()],
            [sg.Button('Red:', size=(8, 1),button_color=('Black', 'Red'),font=("ComicSansMS", 12)), sg.InputText()],
            [sg.Button('Pink:', size=(8, 1),button_color=('Black', 'Pink'),font=("ComicSansMS", 12)), sg.InputText()],
            [sg.Submit(button_color=('Black', 'White'),font=("ComicSansMS", 12)), sg.Cancel(button_color=('White', 'Black'),font=("ComicSansMS", 12))]]

# Create the Window
window = sg.Window('Hello_from_Anja_Gunjan', layout)
event, values = window.read()

window.close()
print('--------------------------------------------')
print('                 Display                    ')
print('--------------------------------------------')

#to display on the terminal
print('Number of Green Objects:', values[0])
numbers[0]=int(values[0])
print('Number of Blue Objects:', values[1])
numbers[1]=int(values[1])
print('Number of Red Objects:', values[2])
numbers[2]=int(values[2])
print('Number of Pink Objects:',values[3])
numbers[3]=int(values[3])

#to run the final code
colors= ['green','blue','red','pink']
count = 0
for i in range(0, len(numbers)):
    count = count + numbers[i]
shopping(colors, numbers, count)